## 量子随机数生成器
*Copyright (c) 2023 Institute for Quantum Computing, Baidu Inc. All Rights Reserved.*

随机数生成器是通过算法、物理信号、环境噪音等来产生随机数列的方法或装置，其在如密码学、机器学习等诸多领域扮演重要角色，它直接影响了密码的可破解性、机器学习模型的泛化性能。经典计算机通过如线性同余法等伪随机算法生成伪随机数，存在一定的缺陷，比如存在周期性等。而量子计算机可以利用量子测量的不确定性生成真随机数，具有不可预测性。本模型库封装了能够调用百度量子云平台上线的百度自研量子计算机、模拟器、第三方硬件生成随机数的方法。

## 生成原理
利用量子计算机可对零初始态 $|0\rangle$ 作用 $H$ 门，得到量子态 $|\psi\rangle$, 
$$
|\psi\rangle = \frac{\sqrt2}{2} |0\rangle + \frac{\sqrt2}{2} |1\rangle .
$$
该量子态 $|\psi\rangle$ 在 Pauli-$Z$ 量子测量（即计算基测量，以 $|0\rangle\langle 0|$，$|1\rangle \langle 1 |$ 为测量算子）下为 $0$ 或为 $1$ 的概率均为 $\frac12$。量子态坍缩的不确定性可以保证得到的 $0$ 或 $1$ 的结果为真随机。重复上述流程即可得到随机比特串，流程示意图如下。
![random_number](./randnum_CN.png)

若实际量子计算机为无噪声理想量子计算机，则测量结果为真随机。不过当前量子计算机均存在一定的噪声，因此我们同时封装了经典的隐私增强算法 [1]，其可以通过利用两台独立的量子计算机（或一台量子计算机的两个独立比特）分别测量得到更长的比特串数据并对两者数据进行后处理提取较短的比特串数据，从而过滤噪声的影响。提取过程示意图如下。
![extractor](./extractor_CN.png)

## 如何使用
### 快速使用
用户可以通过在命令行下输入 `python randnum.py --config config.toml` 即可运行，其中 `config.toml` 为模型配置文件，用户可修改该文件内的参数或输入其它文件实现自定义参数。
### 在线演示
同时，我们这里给出一个在线演示的版本，可以在线进行调试。首先定义配置文件的内容：

In [1]:
example_toml = r'''
# 所需比特串长度
bit_len = 10

# 处理器硬件
backend = 'local_baidu_sim2'        # 本地模拟器
# backend = 'cloud_baidu_qpu_qian'  # 百度自研量子计算机 乾始 （Qian）

# 用户 token，在使用云端服务时需要输入
token = ''

# 是否使用隐私增强后处理，toml 文件中 true 和 false 都应为小写
extract = false

# 信息安全参数，越小则越接近真随机（后处理所需参数，不启动后处理时无需修改）
security = 1e-8

# 处理器硬件 1 和 2 的最小熵（后处理所需参数，不启动后处理时无需修改，取值范围 (0,1)）
min_entr_1 = 0.9 
min_entr_2 = 0.9

# 日志文件路径
log_path = './qrng.log'
'''

接下来执行模型运行过程。

In [2]:
import warnings
import toml
from paddle_quantum.data_analysis.rand_num import random_number_generation
warnings.filterwarnings('ignore')
random_number_generation(**toml.loads(example_toml))

/Applications/anaconda3/envs/pqtest/lib/python3.7/site-packages/paddle/tensor/creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if data.dtype == np.object:


[0, 0, 0, 1, 1, 1, 1, 0, 1, 0]

在这里，我们只需要修改 `example_toml` 字典中的参数，再运行上文全部代码，就可以快速自定义调试。如需使用百度自研量子计算机“乾始”，可将 `backend` 参数的输入修改为 ` 'cloud_baidu_qpu_qian'` （需保留引号，python 字符串数据类型），并输入自己账户 `token`。（若无 `token` 可前往[百度量易伏平台](https://quantum-hub.baidu.com/)注册。)
## 注意事项
- 请保持环境中 `Qcompute` 版本更新，过低版本会导致新上线后端无法识别。后端 `'cloud_baidu_qpu_qian'` 要求 `>= 3.0.0`。
- 使用百度量子云平台的后端时需输入 `token`，请留意个人账户中对应云服务的剩余可用时长。
- 使用云服务时请留意[百度量子云平台](https://quantum-hub.baidu.com/)的各机器运行状态和服务时间。

## 引用信息
```
@article{hayashi2016more,
  title={More efficient privacy amplification with less random seeds via dual universal hash function},
  author={Hayashi, Masahito and Tsurumaru, Toyohiro},
  journal={IEEE Transactions on Information Theory},
  volume={62},
  number={4},
  pages={2213--2232},
  year={2016},
  publisher={IEEE}
}
```
## 参考文献
[1] Hayashi, Masahito, and Toyohiro Tsurumaru. "More efficient privacy amplification with less random seeds via dual universal hash function." IEEE Transactions on Information Theory 62.4 (2016): 2213-2232.